### Import of required libraries and modules

In [1]:
import pandas as pd
import numpy as np
import datetime
import requests
from traffic.data import opensky
from shapely.geometry import Polygon, Point
from traffic.core import Traffic
from traffic.data import airports
from traffic.data.weather import metar
from traffic.data import aircraft
from astral import LocationInfo
from astral.sun import sun
from datetime import timedelta
from matplotlib import pyplot as plt
from scipy.stats import norm


##### Creating an empty table with the required columns

In [ ]:
table = pd.DataFrame(columns=[
    'Flight (Callsign)',
    'Flight ID',
    'ICAO Code',
    'A/C Type',
    'ICAO 24',
    'ICAO Aircraft Type', # Airline
    'Propulsion Type',
    'Number of Engines',
    'MALW [kg]',
    'ICAO Weight Turbulence Category',
    'Speed at TH [kt]',
    'Geoaltitude at TH [ft]',
    'Specific Energy [J/kg]',
    'Energy at MALW [MJ]',
    'Time at TH',
    'Time of Go-around',
    'Hour sin',
    'Hour cos',
    'Minute sin',
    'Minute cos',
    'Day of week sin',
    'Day of week cos',
    'Month sin',
    'Month cos',
    'Day/Night',
    'Time from Previous Landing [s]',
    'Temperature [°C]',
    'Wind speed [kt]',
    'Wind direction [°]',
    'Wind direction sin',
    'Wind direction cos',
    'No Wind',
    'Wind Variable',
    'Headwind [kt]',
    'Crosswind [kt]',
    'Visibility Category',
    'Precipitation [mm]',
    'Meteo idx',
    'Precipitation idx',
    'Arrival Traffic Intensity', # Old version
    'Arrival Traffic Intensity with go-arounds', # Arrival Traffic Intensity 
    'Emergency',
    'Previous Flight',
    'ROT Previous Flight [s]',
    'Average ROT Previous 5 Flight [s]',
    'Average ROT Previous Flight (Intensity) [s]', # Average ROT past 15 min [s]
    'ROT Previous Flight same A/C Type [s]',
    'Average ROT Previous 5 Flights same A/C Type [s]',
    'Average ROT Previous Flight same A/C Type Total [s]',
    'Not Vacated',
    'Short ROT',
    'ROT [s]'])
table

In [ ]:
columns = table.columns
columns

##### Creating a polygon above of rwy 14 in LSZH

In [4]:
# Coordinates of Runway edges
coordinates = [
    (47.482343, 8.536430), #RWY 14 right
    (47.481882, 8.535696), #RWY 14 left
    (47.461058, 8.564121), #RWY 32 left
    (47.461527, 8.564832)  #RWY 32 right
]

#Creating rectangle as polygon
runway_polygon = Polygon(coordinates)

##### Creating a polygon for the vacated area of RWY 14 (Go-Around Polygon)

In [5]:
# Coordinates of Vacated Polygon edges
coordinates2 = [
    (47.467434, 8.555398),  # oben rechts
    (47.466151, 8.552850),  # oben links
    (47.458168, 8.560982),  # unten links
    (47.460002, 8.565674),  # unten rechts
]


#Creating rectangle as polygon
runway_vacated_polygon = Polygon(coordinates2)

##### Creating a polygon after RWY 14

In [6]:
# Coordinates of Go-Around Polygon edges
coordinates3 = [
    (47.461967, 8.566471),  # oben rechts
    (47.460347, 8.563795),  # oben links
    (47.458375, 8.566194),  # unten links
    (47.460188, 8.568865),  # unten rechts
]


#Creating rectangle as polygon
runway_end_polygon = Polygon(coordinates3)

##### Function to correct format of timestamps

In [7]:
def convert_timestamp(x):
    if isinstance(x, (float, int)) and x > 1e9:  # Convert to unix timestamps if float or int.
        return pd.to_datetime(x, unit='s', errors='coerce')
    elif isinstance(x, pd.Timestamp): # If x is alreardy a pandas timestamp, x will be returned
        return x
    else:
        return pd.NaT  # Invalid Value will be converted to Not a Timestamp

##### Define Importdata

In [8]:
start = "2024-06-30 22:00"
stop = "2024-07-31 22:00"
airport_str='LSZH'
go_around_count = 0

##### Import Flightlist

In [9]:
flight_list = opensky.flightlist(start=start, stop=stop, arrival_airport=airport_str)
icao24 = flight_list.icao24

##### Import Flightdata

In [ ]:
timestamps = pd.date_range(start, stop, freq="12h")
data = []
for t1, t2 in zip(timestamps[:-1], timestamps[1:]):
        print(t1, t2)
        tmp = opensky.history(start=str(t1), stop=str(t2), bounds=airports[airport_str].shape.convex_hull.buffer(0.15), icao24=icao24, return_flight=True)
        if tmp is not None:
            data.append(tmp.data)
        
 
trajs = pd.concat(data)
trajs = Traffic(trajs)

##### Correct data format of imported flight data

In [11]:
# Make sure data format is "correct"
trajs.data['timestamp'] = trajs.data['timestamp'].apply(convert_timestamp)
trajs.data['hour'] = trajs.data['hour'].apply(convert_timestamp)
trajs.data['onground'] = trajs.data['onground'].astype(np.bool_)
trajs.data['alert'] = trajs.data['alert'].astype(np.bool_)
trajs.data['spi'] = trajs.data['spi'].astype(np.bool_)
trajs.data['last_position'] = trajs.data['last_position'].astype(np.float64)

trajs.data['icao24'] = trajs.data['icao24'].astype('object')
trajs.data['callsign'] = trajs.data['callsign'].astype('object')
trajs.data['squawk'] = trajs.data['squawk'].astype('object')

##### Assign ID

In [12]:
trajs = trajs.assign_id().eval()

##### Drop duplicates

In [13]:
trajs.data = trajs.data.drop_duplicates(
    subset=["flight_id", "last_position"])

##### Creat a copy of trajs which will be used for go-around classification

In [14]:
trajs_all = Traffic(trajs.data.copy())

##### Remove all data above 4000ft

In [15]:
trajs = trajs.query('geoaltitude < 4000 or onground or geoaltitude.isna()')

##### Resample Data

In [16]:
trajs = trajs.resample('1s').eval()

##### Remove all flights which are not aligned with RWY14

In [ ]:
def keep_aligned_on_rwy_14(flight):
    """Filters flights that are aligned with Runway 14."""
    aligned_segment = next(flight.aligned_on_ils("LSZH", min_duration="30s"), None)
    if aligned_segment is not None and aligned_segment.max("ILS") == "14":
        return flight
    return None

# Use keep_aligned_on_rwy_14 directly within the pipe chain
aligned_flights_lazy = trajs.iterate_lazy().pipe(keep_aligned_on_rwy_14)

# Evaluate and store only flights that meet the condition
trajs = aligned_flights_lazy.eval(max_workers=1)

##### Filter  trajs just for points which are within one of the polygons

In [18]:
# Function that checks if a point is within one of the polygons
def point_in_polygons(row):
    point = Point(row['latitude'], row['longitude'])
    in_runway = runway_polygon.contains(point) or runway_polygon.intersects(point)
    in_vacated = runway_vacated_polygon.contains(point) or runway_vacated_polygon.intersects(point)
    in_end = runway_end_polygon.contains(point) or runway_end_polygon.intersects(point)
    return in_runway or in_vacated or in_end

# Filter the DataFrame using the apply method with lazy execution
trajs.data = trajs.data.loc[trajs.data.apply(point_in_polygons, axis=1)]

##### Function to define Callsign, Flight ID, A/C Type, ICAO 24, Speed, Geoaltitdude, Time at TH and ROT

In [19]:
# Assuming 'table' is an existing DataFrame with the desired columns
altitude_threshold_vacated = 1600  # Altitude limit in feet for runway_vacated_polygon

# Define the processing function for each flight
def process_flight(flight):
    points_inside = []
    points_indside_end = []
    last_valid_altitude = None
    vacated_points_count = 0
    go_around = False
    go_around_time = np.nan
    not_vacated = True
    short_rot = False

    flight = flight.sort_values(by='timestamp')

    # Iterate through flight data to get latitude, longitude, and altitude
    for _, row in flight.data.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        alt = row['altitude']

        if np.isnan(alt) and last_valid_altitude is not None:
            alt = last_valid_altitude

        point = Point(lat, lon)

        if runway_polygon.contains(point) or runway_polygon.intersects(point):
            points_inside.append(row)
            last_valid_altitude = alt

        if runway_vacated_polygon.contains(point) or runway_vacated_polygon.intersects(point) and alt <= altitude_threshold_vacated:
            vacated_points_count += 1

        if runway_end_polygon.contains(point) or runway_end_polygon.intersects(point):
            points_indside_end.append(row)

        if vacated_points_count >= 5:
            not_vacated = False
            reason = 'Not Vacated'
            break

    flight_data_rwy = pd.DataFrame(points_inside)
    flight_rwy_end_data = pd.DataFrame(points_indside_end)
    if not flight_data_rwy.empty:
        first_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].min())
        last_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].max())
        rot = (last_timestamp - first_timestamp).total_seconds()

        time_th = 150
        time_min = 40

        if rot < time_min:
            short_rot = True
            points_inside = []
            points_indside_end = []
            last_valid_altitude = None
            for _, row in flight.data.iterrows():
                lat = row['latitude']
                lon = row['longitude']
                alt = row['altitude']

                if np.isnan(alt) and last_valid_altitude is not None:
                    alt = last_valid_altitude

                point = Point(lat, lon)

                if runway_polygon.contains(point) or runway_polygon.intersects(point):
                    points_inside.append(row)
                    last_valid_altitude = alt

                if runway_vacated_polygon.contains(point) or runway_vacated_polygon.intersects(point) and alt <= altitude_threshold_vacated:
                    vacated_points_count += 1

                if runway_end_polygon.contains(point) or runway_end_polygon.intersects(point):
                    points_indside_end.append(row)

            flight_data_rwy = pd.DataFrame(points_inside)
            flight_rwy_end_data = pd.DataFrame(points_indside_end)

        if not flight_data_rwy.empty:
            first_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].min())
            last_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].max())
            rot = (last_timestamp - first_timestamp).total_seconds()

        has_end_data = not flight_rwy_end_data.empty
        points_above_1800 = (flight_rwy_end_data['geoaltitude'] > 1800).sum() if has_end_data else 0

        if (
            ((rot > time_th or has_end_data) and trajs_all[flight.flight_id].has('go_around("LSZH")')) or 
            (has_end_data and points_above_1800 >= 1)
        ):

            go_around = True
            flight_data_rwy = flight_data_rwy.sort_values(by='timestamp')
            go_around_time = flight_data_rwy.iloc[0]['timestamp']
            flight_data_rwy['time_diff'] = flight_data_rwy['timestamp'].diff().dt.total_seconds()
            gap_index = flight_data_rwy[flight_data_rwy['time_diff'] > 150].index
            flight_data_rwy = flight_data_rwy.drop(columns=['time_diff'])

            if gap_index.empty:
                return {
                    'Flight (Callsign)': flight.callsign,
                    'Flight ID': flight.flight_id,
                    'Time of Go-around': go_around_time,
                }

            if not gap_index.empty:
                # Set a maximum iteration counter to prevent infinite loops
                max_iterations = 10
                iteration_count = 0
                while ((rot > time_th) and (iteration_count < max_iterations)):
                    iteration_count += 1
                    flight_data_rwy['time_diff'] = flight_data_rwy['timestamp'].diff().dt.total_seconds()
                    gap_index = flight_data_rwy[flight_data_rwy['time_diff'] > 150].index
                    if gap_index.empty:
                        break
                    first_gap_index = gap_index[0]
                    flight_data_rwy = flight_data_rwy.loc[first_gap_index:]
                    if flight_data_rwy.empty:
                        return {
                            'Flight (Callsign)': flight.callsign,
                            'Flight ID': flight.flight_id,
                            'Time of Go-around': go_around_time,
                        }
                    flight_data_rwy = flight_data_rwy.drop(columns=['time_diff'])
                    first_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].min())
                    last_timestamp = pd.to_datetime(flight_data_rwy['timestamp'].max())
                    rot = (last_timestamp - first_timestamp).total_seconds()

        # Collect relevant data to return as a row
        return {
            'Flight (Callsign)': flight.callsign,
            'Flight ID': flight.flight_id,
            'A/C Type': flight.typecode,
            'icao24': flight.icao24,
            'Speed at TH [kt]': flight_data_rwy.iloc[0]['groundspeed'],
            'Geoaltitude at TH [ft]': flight_data_rwy.iloc[0]['geoaltitude'],
            'Time at TH': flight_data_rwy.iloc[0]['timestamp'],
            'Go-around': go_around,
            'Time of Go-around': go_around_time,
            'Emergency': flight.emergency().has(),
            'Not Vacated': not_vacated,
            'Short ROT': short_rot,
            'ROT [s]': rot
        }

    return None

##### Evaluate Function on Trajs and add data to table

In [20]:
# Lazy iteration over flights using iterate_lazy
table = trajs.iterate_lazy().pipe(process_flight).eval()

##### Sort Table in regards of Time at TH

In [ ]:
# Sort values
table = table.sort_values(by='Time at TH')

# Reset index
table.reset_index(drop=True, inplace=True)

table


##### Extract and remove Go-Arounds and Go-Around with RWY changes from table

In [22]:
rwy_change = table.loc[table['ROT [s]'].isna()]
go_around_list = table.dropna(subset=['Time of Go-around'])
table = table.dropna(subset=['ROT [s]'])
table.reset_index(drop=True, inplace=True)

##### Remove Corrupt Flights

In [23]:
delete_list = [
    'ETD39B_31068',
    'FJO649_27511',
    'TAG18_27961',
    'THY62PE_25403',
    'SWR3BY_14205',
    'SWR2EP_15155',
    'GPX600_7051',
]

table = table[~table['Flight ID'].isin(delete_list)]

table.reset_index(drop=True, inplace=True)

### Meteo

##### Define Function to find the closest METAR

In [24]:
# Function to find the closest METAR to a timestamp of a flight in the past or at the same time
def find_closest_past_metar(timestamp, meteo_times):
    # Calculate time difference
    time_diff = meteo_times - timestamp
    # Filter out future METAR times (i.e., positive time differences)
    past_times = time_diff[time_diff <= pd.Timedelta(0)]
    
    # If there are no past times available, return None or handle accordingly
    if past_times.empty:
        return None

    # Calculate absolute differences and find the closest
    closest_index = past_times.abs().idxmin()  # Find closest index from past times
    return closest_index

##### Import of all METARS within Time periode

In [ ]:
meteo_data = []
for t1, t2 in zip(timestamps[:-1], timestamps[1:]):
        print(t1, t2)
        tmp = metar.METAR('LSZH').get(
             start=str(t1),
             stop=str(t2),
        )
        if tmp is not None:
            meteo_data.append(tmp)       
 
meteo = pd.concat(meteo_data)

# Sort the DataFrame by time and reset the index
meteo = meteo.sort_values(by="time").reset_index(drop=True)

##### Add weather data for each flight to the table based on the closest METAR

In [26]:
# Loop through each row in the DataFrame
for flight in range(len(table)):
    # Convert the time column to datetime objects
    time = pd.to_datetime(table['Time at TH'][flight])
    
    # Find the index of the closest METAR report
    idx = find_closest_past_metar(time, meteo['time'])
    
    # Get the corresponding values from the METAR DataFrame
    wind_spd = meteo.wind_speed[idx]
    wind_dir = meteo.wind_dir[idx]
    visibility = meteo.vis[idx]
    temperature = meteo.temp[idx]
    
    # Insert the values into the corresponding columns in 'table'
    table.at[flight, 'Wind speed [kt]'] = wind_spd
    table.at[flight, 'Wind direction [°]'] = wind_dir
    table.at[flight, 'Visibility Category'] = visibility
    table.at[flight, 'Temperature [°C]'] = temperature
    table.at[flight, 'Meteo idx'] = idx


##### Convert Temperature, Wind speed and Wind direction Data in table to numerical values

In [27]:
# Convert all values in the relevant columns to strings
table['Temperature [°C]'] = table['Temperature [°C]'].astype(str)
table['Wind speed [kt]'] = table['Wind speed [kt]'].astype(str)
table['Wind direction [°]'] = table['Wind direction [°]'].astype(str)

# Clean the columns (e.g., remove leading/trailing spaces)
table['Temperature [°C]'] = table['Temperature [°C]'].str.strip()
table['Wind speed [kt]'] = table['Wind speed [kt]'].str.strip()
table['Wind direction [°]'] = table['Wind direction [°]'].str.strip()

# Replace empty strings or 'None' with NaN in the 'Wind direction [°]' column
table['Wind direction [°]'] = table['Wind direction [°]'].replace(['None', ''], pd.NA)

# Extract the numeric values
table['Temperature [°C]'] = table['Temperature [°C]'].str.extract(r'(\d+)')
table['Wind speed [kt]'] = table['Wind speed [kt]'].str.extract(r'(\d+)')
table['Wind direction [°]'] = table['Wind direction [°]'].str.extract(r'(\d+)')

# Convert the extracted values to floats to display NaN for missing values
table['Temperature [°C]'] = pd.to_numeric(table['Temperature [°C]'], errors='coerce').astype(float)
table['Wind speed [kt]'] = pd.to_numeric(table['Wind speed [kt]'], errors='coerce').astype(float)
table['Wind direction [°]'] = pd.to_numeric(table['Wind direction [°]'], errors='coerce').astype(float)


##### Define Visibility categories

In [28]:
# Function to categorize visibility into categories
def categorize_visibility(visibility):
    if visibility <= 1000:
        return 0  # Very poor
    elif visibility <= 2000:
        return 1  # Poor
    elif visibility <= 3000:
        return 2
    elif visibility <= 4000:
        return 3
    elif visibility <= 5000:
        return 4
    elif visibility <= 6000:
        return 5
    elif visibility <= 7000:
        return 6
    elif visibility <= 8000:
        return 7
    elif visibility <= 9000:
        return 8
    elif visibility <= 10000:
        return 9  # Good
    else:
        return 10  # Very good

##### Transfer Visibility Data in table to visibility category

In [29]:
# Convert all values in the 'Visibility Category' column to strings
table['Visibility Category'] = table['Visibility Category'].astype(str)

# Clean the 'Visibility Category' column (e.g., remove leading/trailing spaces)
table['Visibility Category'] = table['Visibility Category'].str.strip()

# Replace empty strings or 'None' with NaN for better compatibility with ML
table['Visibility Category'] = table['Visibility Category'].replace(['None', ''], pd.NA)

# Conditional processing: If 'greater than' is present, set the value to '12000',
# otherwise extract the number as an integer or float
table['Visibility Category'] = table['Visibility Category'].apply(
    lambda x: 12000 if isinstance(x, str) and 'greater than' in x.lower() else x
)

# For all non-'greater than' values, extract the number and keep it as a float (NaN for missing)
table['Visibility Category'] = table['Visibility Category'].apply(
    lambda x: pd.to_numeric(x.split()[0], errors='coerce') if isinstance(x, str) else x
)

# Ensure the final column is numeric with NaN for missing values
table['Visibility Category'] = pd.to_numeric(table['Visibility Category'], errors='coerce')


# Apply the visibility categorization directly to the 'Visibility Category' column
table['Visibility Category'] = table['Visibility Category'].apply(categorize_visibility)


##### Import Precipitation

In [30]:
# Coordinates for Zurich Airport RWY 14
latitude = 47.473626
longitude = 8.547640

# Convert start and stop times to datetime format
start_datetime = pd.to_datetime(start)
stop_datetime = pd.to_datetime(stop)

# Extract the date in "YYYY-MM-DD" format
start_date = start_datetime.strftime("%Y-%m-%d")
end_date = stop_datetime.strftime("%Y-%m-%d")
hourly = "precipitation"

# Fetch weather data from Open-Meteo
url = f"https://archive-api.open-meteo.com/v1/era5?latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&hourly={hourly}"
response = requests.get(url)
data = response.json()

# Create a Pandas DataFrame with timestamps and precipitation data
meteo_df = pd.DataFrame({
    'Timestamp': pd.to_datetime(data['hourly']['time']),
    'Precipitation [mm]': data['hourly']['precipitation']
})

# Optional: Ensure the timestamps are localized to UTC if needed
meteo_df['Timestamp'] = meteo_df['Timestamp'].dt.tz_localize('UTC')


##### Define function to find clostes Precipitation Data

In [31]:
# Function to determine the nearest precipitation value to the flight timestamp in the past or at the same time
def find_closest_past_precipitation(flight_time, meteo_df):
    # Filter out future precipitation times (i.e., only keep timestamps in the past or at the same time)
    past_times_df = meteo_df[meteo_df['Timestamp'] <= flight_time]

    # If there are no past times available, return None
    if past_times_df.empty:
        return None

    # Calculate the time differences
    time_diff = (past_times_df['Timestamp'] - flight_time).abs()

    # Find the index of the minimum time difference (i.e., closest past time)
    closest_index = time_diff.idxmin()
    
    return closest_index

##### Add closest Precipitation Data to table

In [ ]:
# Loop through each flight time in 'table' to find the closest precipitation value
for flight in range(len(table)):
    # Temporary conversion of 'Time at TH' to datetime without changing the original table format
    flight_time = pd.to_datetime(table['Time at TH'].iloc[flight], errors='coerce')
    
    # Check if the conversion was successful
    if pd.notna(flight_time):  # Valid datetime object
        # Find the index of the closest precipitation data in the past or at the same time
        idx = find_closest_past_precipitation(flight_time, meteo_df)

        if idx is not None:
            # Get the corresponding precipitation value and timestamp
            precipitation_value = meteo_df.loc[idx, 'Precipitation [mm]']
            corresponding_timestamp = meteo_df.loc[idx, 'Timestamp']

            # Insert the precipitation value and timestamp into the corresponding columns in 'table'
            table.at[flight, 'Precipitation [mm]'] = precipitation_value
            table.at[flight, 'Precipitation idx'] = idx
            table.at[flight, 'Precipitation Timestamp'] = corresponding_timestamp
        else:
            # No valid past time found, set value to None
            table.at[flight, 'Precipitation [mm]'] = None
            table.at[flight, 'Precipitation idx'] = None
            table.at[flight, 'Precipitation Timestamp'] = None

    else:
        # Handle cases where datetime conversion fails
        table.at[flight, 'Precipitation [mm]'] = None
        table.at[flight, 'Precipitation idx'] = None
        table.at[flight, 'Precipitation Timestamp'] = None

# Display the updated table
table


### Import and Add Data from FAA Aircraft Characteristics Database

##### In case of missing data in opensky, an manually built database is used to add ICAO Aircraft Type

In [33]:
# Load the list from the CSV file into a DataFrame
DF_MISSING_ICAO24 = pd.read_csv('missing_icao24_data.csv')

##### Add and check A/C Type for missing  or outdated entries by using a manual database

In [ ]:
# Create a dictionary from the second table (DF_MISSING_ICAO24) for quick access
icao24_dict = DF_MISSING_ICAO24.set_index('icao24').to_dict(orient='index')

# Update 'A/C Type' for missing or outdated values
for icao24, info in icao24_dict.items():
    # Mask for rows with matching icao24 and either missing or outdated 'A/C Type'
    mask_ac_type = (table['icao24'] == icao24) & ((table['A/C Type'].isna()) | (table['A/C Type'] == '') | (table['A/C Type'] != info['typecode']))
    table.loc[mask_ac_type, 'A/C Type'] = info['typecode']

# Display the result for 'A/C Type' updates
table

##### Load the FAA Characterisitcs Database Excel file into a DataFrame

In [35]:
aircraft_characteristics = pd.read_excel('FAA-Aircraft-Char-DB-AC-150-5300-13B-App-2023-09-07.xlsx')

##### Add Maximum Allowable Landing Weight (MALW) anc ICAO Weight Turbulence Category (WTC) to table

In [36]:
table = table.merge(aircraft_characteristics[['ICAO_Code', 'MALW_lb', 'ICAO_WTC']],
                                  left_on='A/C Type',    # Column in table
                                  right_on='ICAO_Code',  # Column in aircraft_characteristics
                                  how='left').drop(columns='ICAO_Code')  # Optionally drop 'ICAO_Code' after merge


##### Convert to metric units

In [37]:
# Convert non-NaN values from pounds to kilograms and round, keeping NaNs as they are
table['MALW [kg]'] = table['MALW_lb'].apply(lambda x: round(x * 0.453592) if pd.notna(x) else x)

### Engineer further Data from table

##### Add ICAO Aircraft Type table

In [38]:
# Perform the merge
table = table.merge(aircraft.data[['icao24', 'icaoaircrafttype']],
                                  on='icao24',
                                  how='left')

##### In case of missing data in opensky, an manually built database is used to add ICAO Aircraft Type

In [ ]:
# Update 'icaoaircrafttype' for missing or outdated values
for icao24, info in icao24_dict.items():
    # Mask for rows with matching icao24 and either missing or outdated 'icaoaircrafttype'
    mask_icaoaircrafttype = (table['icao24'] == icao24) & ((table['icaoaircrafttype'].isna()) | (table['icaoaircrafttype'] == '') | (table['icaoaircrafttype'] != info['icaoaircrafttype']))
    table.loc[mask_icaoaircrafttype, 'icaoaircrafttype'] = info['icaoaircrafttype']

# Display the result for 'icaoaircrafttype' updates
table

##### Extract Number of Engines and Propulsion Type from ICAO Aircraft Code

In [ ]:
# Function to extract number of engines and propulsion type dynamically based on length
def get_engine_info(icao_type):
    if isinstance(icao_type, str) and len(icao_type) == 3:  # Check if the length is exactly 3
        num_engines = int(icao_type[1])  # Extract number of engines (2nd character) and convert to integer
        propulsion_code = icao_type[2]   # Extract propulsion type (3rd character)

        # Check for propulsion type based on the 3rd character
        if propulsion_code == 'J':
            propulsion_type = 'Jet'
        elif propulsion_code == 'T':
            propulsion_type = 'Turboprop'
        elif propulsion_code == 'P':
            propulsion_type = 'Piston'
        else:
            propulsion_type = np.nan  # Handle unknown propulsion types

        return propulsion_type, num_engines
    else:
        return np.nan, np.nan  # Handle cases where length is not exactly 3

# Apply the function to the 'icaoaircrafttype' column
table['Propulsion Type'], table['Number of Engines'] = zip(
    *table['icaoaircrafttype'].apply(get_engine_info)
)

# Ensure NaN instead of <NA> in the 'Number of Engines' column
table['Number of Engines'] = table['Number of Engines'].replace({pd.NA: np.nan})

# Convert "Number of Engines" to integer type
table['Number of Engines'] = table['Number of Engines'].astype('float64')  # Use float to handle NaN

# Display the result
table


##### Derive ICAO Code from callsign (Airline)

In [41]:
# Step 1: Extract the first three characters of the 'Flight (Callsign)' column to get the ICAO Code
table['ICAO Code'] = table['Flight (Callsign)'].str[:3]

# Step 2: Define a threshold for the minimum number of occurrences required for an ICAO Code to be considered valid
threshold = 15  # Set your desired threshold value here

# Step 3: Count occurrences of each ICAO Code in the table
icao_counts = table['ICAO Code'].value_counts()

# Step 4: Assign "Unknown" to ICAO Codes that do not meet the threshold
table['ICAO Code'] = table['ICAO Code'].apply(lambda x: x if icao_counts.get(x, 0) >= threshold else 'Unknown')


##### Cyclic encoding of Time at TH

In [42]:
# Sample structure based on your 'Time at TH' column
table['Hour'] = table['Time at TH'].dt.hour
table['Minute'] = table['Time at TH'].dt.minute
table['Day of week'] = table['Time at TH'].dt.weekday
table['Month'] = table['Time at TH'].dt.month
table['Year'] = table['Time at TH'].dt.year % 100  # Using modulo 100 for 'year' cyclic encoding

# Function to apply encoding to one column
def cyclic_enc(df, feature, max_val):
    df[feature + " sin"] = (
        0.5 * np.sin(2 * np.pi * df[feature] / max_val) + 0.5
    )
    df[feature + " cos"] = (
        0.5 * np.cos(2 * np.pi * df[feature] / max_val) + 0.5
    )
    return df

def encode_df(df):
    # Cyclic encoding of hour, minute, weekday, month, and year
    df = cyclic_enc(df, 'Hour', 24)
    df = cyclic_enc(df, 'Minute', 60)
    df = cyclic_enc(df, 'Day of week', 7)
    df = cyclic_enc(df, 'Month', 12)
    df = cyclic_enc(df, 'Year', 100)
    
    # Drop original columns after encoding
    df = df.drop(columns=['Hour', 'Minute', 'Day of week', 'Month', 'Year'])
    return df

# Apply encoding to the dataset
table = encode_df(table)


##### Determine Day or Night Status Based on Sun's Position at 'Time at TH'

In [43]:
# Define location information (Zurich Airport)
location = LocationInfo("Zurich Airport", "Switzerland", "Europe/Zurich", latitude, longitude)

# Function to determine if it is day or night based on the timestamp
def is_daytime(timestamp):
    # Calculate sunrise and sunset times for the date of the timestamp
    s = sun(location.observer, date=timestamp.date())
    sunrise = s['sunrise']
    sunset = s['sunset']
    
    # Check if the timestamp is between sunrise and sunset
    return sunrise <= timestamp <= sunset

# Apply the function to the 'Time at TH' column and create a new 'Day/Night' column
table['Day/Night'] = table['Time at TH'].apply(lambda x: 'Day' if is_daytime(x) else 'Night')


##### Calculate Specific Energy basen on Altitude and Speed at TH

In [44]:
# Airport elevation in feet
airport_elevation_ft = 1417

# Constants
g = 9.81  # Acceleration due to gravity in m/s²

# Calculate relative altitude in meters without storing intermediate results in the table
relative_altitude_m = (table['Geoaltitude at TH [ft]'] - airport_elevation_ft) * 0.3048  # Convert ft to m
speed_m_s = table['Speed at TH [kt]'] * 0.514444  # Convert knots to m/s

# Calculate specific energy (energy per unit mass) and preserve NaN values
table['Specific Energy [J/kg]'] = (
    g * relative_altitude_m + 0.5 * speed_m_s ** 2
).round(0).astype('Int64')


##### Calculate Energy at MALW basen on Specific Energy at TH

In [45]:
table['Energy at MALW [MJ]'] = table['MALW [kg]']*table['Specific Energy [J/kg]']/1e6

##### Determine Time at TH Difference between succesive landings

In [46]:
# Convert 'Time at TH' to datetime format and force conversion with 'coerce' to mark non-date values as NaT
table['Time at TH'] = pd.to_datetime(table['Time at TH'], errors='coerce')

# Check if all values were correctly converted
if table['Time at TH'].isnull().any():
    print("Warning: Some values in 'Time at TH' could not be converted and were marked as NaT.")

# Calculate the time difference to the previous landing in seconds and assign it to a new column
table['Time from Previous Landing [s]'] = (
    table['Time at TH'] - table['Time at TH'].shift(1)
).dt.total_seconds()

# Fill NaN values in the 'Time from Previous Landing [s]' column (e.g., for the first row) with 0 or a placeholder
table['Time from Previous Landing [s]'] = table['Time from Previous Landing [s]'].fillna(0)


##### Correct Time at TH Difference between succesive landings for go arounds

In [47]:
# Loop through each go-around time to find the next closest landing entry in 'table'
for go_around_time in go_around_list['Time of Go-around']:
    # Find the entry in 'table' that has the closest time greater than the go-around time
    subsequent_entries = table[table['Time at TH'] > go_around_time]
    
    # If there are no subsequent entries, skip this go-around time
    if subsequent_entries.empty:
        continue

    # Find the next landing time after the go-around
    closest_index = subsequent_entries['Time at TH'].idxmin()

    # Calculate the time difference in seconds from the go-around time to the next 'Time at TH'
    time_diff = (table.loc[closest_index, 'Time at TH'] - go_around_time).total_seconds()

    # Update the 'Time from Previous Landing [s]' for the next landing
    table.loc[closest_index, 'Time from Previous Landing [s]'] = time_diff


##### Add additional Wind data and cyclic encode wind data

In [ ]:
# 1. `No Wind`: Wind speed is 0
table['No Wind'] = table['Wind speed [kt]'] == 0

# 2. `Wind Variable`: Wind direction is NaN, but wind speed > 0
table['Wind Variable'] = table.apply(
    lambda row: pd.isna(row['Wind direction [°]']) and row['Wind speed [kt]'] > 0,
    axis=1
)

# 3. Transform wind direction into sine/cosine
# Process only valid wind directions
table['Wind direction sin_raw'] = np.sin(np.deg2rad(table['Wind direction [°]']))
table['Wind direction cos_raw'] = np.cos(np.deg2rad(table['Wind direction [°]']))

# 4. Scale sine/cosine values to the range [0, 1]
table['Wind direction sin'] = (table['Wind direction sin_raw'] + 1) / 2
table['Wind direction cos'] = (table['Wind direction cos_raw'] + 1) / 2

# Set sine/cosine values to 0 for "No Wind" and "Wind Variable"
table.loc[table['No Wind'] | table['Wind Variable'], ['Wind direction sin', 'Wind direction cos']] = 0

# Remove unnecessary intermediate columns
table.drop(columns=['Wind direction sin_raw', 'Wind direction cos_raw'], inplace=True)

table


##### Determine Head- and Crosswind Components based on Wind Data

In [ ]:
runway_direction = airports['LSZH'].runways.data.bearing.loc[2]

# Convert runway direction to radians
runway_rad = np.radians(runway_direction)

# Calculate and round Headwind and Crosswind to two decimal places
table['Headwind [kt]'] = table.apply(
    lambda row: round(row['Wind speed [kt]'] * np.cos(np.radians(row['Wind direction [°]']) - runway_rad), 2) 
    if pd.notna(row['Wind speed [kt]']) and pd.notna(row['Wind direction [°]']) else np.nan, axis=1
)

table['Crosswind [kt]'] = table.apply(
    lambda row: round(row['Wind speed [kt]'] * np.sin(np.radians(row['Wind direction [°]']) - runway_rad), 2) 
    if pd.notna(row['Wind speed [kt]']) and pd.notna(row['Wind direction [°]']) else np.nan, axis=1
)
# Display the updated DataFrame to confirm the new columns
table


##### Determine Arrival Traffic Intensity (Old Version)

In [50]:
# Define the time interval
time_interval = 15
time_interval = pd.Timedelta(minutes=15)

# Add a new column to store the count of landings within the defined interval
table['Arrival Traffic Intensity'] = 0

# Loop through each row to count previous landings within the time interval
for i in range(len(table)):
    # Calculate the start of the time window based on the current 'Time at TH' and the defined interval
    window_start = table['Time at TH'].iloc[i] - time_interval
    
    # Count all landings that occurred within the time window before the current 'Time at TH'
    count = table[(table['Time at TH'] >= window_start) & (table['Time at TH'] < table['Time at TH'].iloc[i])].shape[0]
    
    # Store the count in the new column 'Landings_in_Interval'
    table.at[i, 'Arrival Traffic Intensity'] = count


##### Determine Arrival Traffic Intensity with Go Arounds (Arrival Traffic Intensity)

In [51]:
# Define the time interval
time_interval = pd.Timedelta(minutes=15)

# Add a new column to store the count of landings within the defined interval
table['Arrival Traffic Intensity with go-arounds'] = 0

# Concatenate table and go_around_list to have all relevant times (regular landings and go-arounds)
combined_times = pd.concat([
    table[['Time at TH']].rename(columns={'Time at TH': 'Time'}),
    go_around_list[['Time of Go-around']].rename(columns={'Time of Go-around': 'Time'})
], axis=0).sort_values(by='Time').reset_index(drop=True)

# Loop through each row in table to count previous landings and go-arounds within the time interval
for i in range(len(table)):
    # Get the current 'Time at TH'
    current_time = table['Time at TH'].iloc[i]
    
    # Calculate the start of the time window based on the current 'Time at TH' and the defined interval
    window_start = current_time - time_interval
    
    # Count all landings and go-arounds that occurred within the time window before the current 'Time at TH'
    count = combined_times[(combined_times['Time'] >= window_start) & (combined_times['Time'] < current_time)].shape[0]
    
    # Store the count in the new column 'Arrival Traffic Intensity with go-arounds'
    table.at[i, 'Arrival Traffic Intensity with go-arounds'] = count

##### Add Flight ID of previous flight to table

In [52]:
for idx in range(1, len(table)):
    table.loc[idx, 'Previous Flight'] = table.loc[idx-1, 'Flight ID']


### Determine ROT trends of previous flights

In [53]:
# Example buffer time in seconds
time_buffer = 0

# Initialize new columns for the calculated metrics
table['ROT Previous Flight [s]'] = None
table['Average ROT Previous 5 Flight [s]'] = None
table['Average ROT Previous Flight (Intensity) [s]'] = None
table['ROT Previous Flight same A/C Type [s]'] = None
table['Average ROT Previous 5 Flights same A/C Type [s]'] = None
table['Average ROT Previous Flight same A/C Type Total [s]'] = None

# Perform calculations based on clarified column definitions
for i in range(len(table)):
    # Get the ROT for the previous flight, ensuring sufficient time gap
    for j in range(i - 1, -1, -1):
        if table.loc[i, 'Time from Previous Landing [s]'] > table.loc[j, 'ROT [s]'] + time_buffer:
            table.loc[i, 'ROT Previous Flight [s]'] = table.loc[j, 'ROT [s]']
            break

    # Calculate the average ROT for the previous 5 flights, ensuring sufficient time gap
    valid_rot_values = []
    for j in range(i - 1, -1, -1):
        if table.loc[i, 'Time from Previous Landing [s]'] > table.loc[j, 'ROT [s]'] + time_buffer:
            valid_rot_values.append(table.loc[j, 'ROT [s]'])
        if len(valid_rot_values) == 5:
            break
    if len(valid_rot_values) > 0:
        table.loc[i, 'Average ROT Previous 5 Flight [s]'] = sum(valid_rot_values) / len(valid_rot_values)

    # Calculate the average ROT based on arrival traffic intensity, ensuring sufficient time gap
    intensity = table.loc[i, 'Arrival Traffic Intensity']
    valid_rot_values_intensity = []
    for j in range(i - 1, -1, -1):
        if table.loc[i, 'Time from Previous Landing [s]'] > table.loc[j, 'ROT [s]'] + time_buffer:
            valid_rot_values_intensity.append(table.loc[j, 'ROT [s]'])
        if len(valid_rot_values_intensity) == intensity:
            break
    if len(valid_rot_values_intensity) > 0:
        table.loc[i, 'Average ROT Previous Flight (Intensity) [s]'] = sum(valid_rot_values_intensity) / len(valid_rot_values_intensity)

    # Get the ROT for the previous flight with the same A/C type, ensuring sufficient time gap
    previous_same_ac_type = table[(table.index < i) & (table['A/C Type'] == table.loc[i, 'A/C Type'])]
    for idx in reversed(previous_same_ac_type.index):
        if table.loc[i, 'Time from Previous Landing [s]'] > table.loc[idx, 'ROT [s]'] + time_buffer:
            table.loc[i, 'ROT Previous Flight same A/C Type [s]'] = table.loc[idx, 'ROT [s]']
            break

    # Calculate the average ROT for the previous 5 flights with the same A/C type, ensuring sufficient time gap
    valid_rot_values_same_ac_type = []
    for idx in reversed(previous_same_ac_type.index):
        if table.loc[i, 'Time from Previous Landing [s]'] > table.loc[idx, 'ROT [s]'] + time_buffer:
            valid_rot_values_same_ac_type.append(table.loc[idx, 'ROT [s]'])
        if len(valid_rot_values_same_ac_type) == 5:
            break
    if len(valid_rot_values_same_ac_type) > 0:
        table.loc[i, 'Average ROT Previous 5 Flights same A/C Type [s]'] = sum(valid_rot_values_same_ac_type) / len(valid_rot_values_same_ac_type)

    # Calculate the total average ROT of all previous flights with the same A/C type, ensuring sufficient time gap
    valid_rot_values_total_same_ac_type = []
    for idx in previous_same_ac_type.index:
        if table.loc[i, 'Time from Previous Landing [s]'] > table.loc[idx, 'ROT [s]'] + time_buffer:
            valid_rot_values_total_same_ac_type.append(table.loc[idx, 'ROT [s]'])
    if len(valid_rot_values_total_same_ac_type) > 0:
        table.loc[i, 'Average ROT Previous Flight same A/C Type Total [s]'] = sum(valid_rot_values_total_same_ac_type) / len(valid_rot_values_total_same_ac_type)


##### Rename the columns

In [54]:
table = table.rename(columns={
    'icao24': 'ICAO 24',
    'icaoaircrafttype': 'ICAO Aircraft Type',
    'ICAO_WTC': 'ICAO Weight Turbulence Category'}
)

In [ ]:
table.columns

### Rearrange the columns as definded at the beginning

In [ ]:
table = table[columns]
table

##### Create a table sorted after maximum ROT time

In [ ]:
table_rot_max = table.sort_values(by='ROT [s]', ascending=False)

table_rot_max

##### Create a table sorted after minimum ROT time

In [ ]:
table_rot_min = table.sort_values(by='ROT [s]', ascending=True)

table_rot_min

In [ ]:
plt.hist(table['ROT [s]'], bins=range(30, 121), edgecolor='black')

plt.title('Histogram of ROT Values')
plt.xlabel('ROT [s]')
plt.ylabel('Number of occurrences')
plt.grid()

### Save Table

In [60]:
table.to_csv('final_table.csv')